In [1]:
import polars as pl
pl.Config.with_columns_kwargs = True

In [14]:
pitching = pl.read_csv('./Data/lahman_1871-2023_csv/Pitching.csv')
pitching.head()

playerID,yearID,stint,teamID,lgID,W,L,G,GS,CG,SHO,SV,IPouts,H,ER,HR,BB,SO,BAOpp,ERA,IBB,WP,HBP,BK,BFP,GF,R,SH,SF,GIDP
str,i64,i64,str,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,f64,f64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
"""aardsda01""",2004,1,"""SFN""","""NL""",1,0,11,0,0,0,0,32,20,8,1,10,5,0.417,6.75,0,0,2,0,61,5,8,0,1,1
"""aardsda01""",2006,1,"""CHN""","""NL""",3,0,45,0,0,0,0,159,41,24,9,28,49,0.214,4.08,0,1,1,0,225,9,25,1,3,2
"""aardsda01""",2007,1,"""CHA""","""AL""",2,1,25,0,0,0,0,97,39,23,4,17,36,0.3,6.4,3,2,1,0,151,7,24,2,1,1
"""aardsda01""",2008,1,"""BOS""","""AL""",4,2,47,0,0,0,0,146,49,30,4,35,49,0.268,5.55,2,3,5,0,228,7,32,3,2,4
"""aardsda01""",2009,1,"""SEA""","""AL""",3,6,73,0,0,0,38,214,49,20,4,34,80,0.19,2.52,3,2,0,0,296,53,23,2,1,2


In [36]:
league_era = (
    pitching
    .select(['yearID', 'lgID', 'IPouts', 'R'])
    .filter(pl.col('yearID') >= 1946)
    .group_by([pl.col('yearID'), pl.col('lgID')])
    .agg(pl.col('R').sum().alias('total_runs'),
         pl.col('IPouts').sum().alias('total_outs'))
    .with_columns(era = (pl.col('total_runs') / pl.col('total_outs')) * 27)
    .with_columns(
        pl.when(pl.col('yearID') >= 1973)
          .then(pl.when(pl.col('lgID') == 'AL')
                  .then(pl.lit('yes'))
                  .otherwise(pl.lit('no'))
               )
          .otherwise(pl.lit('no'))
          .alias('DH')
    )
)

league_era


yearID,lgID,total_runs,total_outs,era,DH
i64,str,i64,i64,f64,str
2011,"""AL""",10036,60850,4.453114,"""yes"""
1951,"""NL""",5552,33462,4.479828,"""no"""
1962,"""NL""",7278,43470,4.520497,"""no"""
2001,"""NL""",12186,69223,4.753073,"""no"""
1961,"""AL""",7342,43280,4.580268,"""no"""
…,…,…,…,…,…
1975,"""AL""",8281,51821,4.314602,"""yes"""
1982,"""NL""",7947,52630,4.076933,"""no"""
1978,"""NL""",7742,52000,4.019885,"""no"""


In [13]:
batting = pl.read_csv('./Data/lahman_1871-2023_csv/Batting.csv')
batting.head()

playerID,yearID,stint,teamID,lgID,G,G_batting,AB,R,H,2B,3B,HR,RBI,SB,CS,BB,SO,IBB,HBP,SH,SF,GIDP,G_old
str,i64,i64,str,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,str
"""aardsda01""",2004,1,"""SFN""","""NL""",11,null,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,null
"""aardsda01""",2006,1,"""CHN""","""NL""",45,null,2,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,null
"""aardsda01""",2007,1,"""CHA""","""AL""",25,null,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,null
"""aardsda01""",2008,1,"""BOS""","""AL""",47,null,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,null
"""aardsda01""",2009,1,"""SEA""","""AL""",73,null,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,null
